In [14]:
from typing import Callable
import numpy as np
import pandas as pd
pd.options.display.expand_frame_repr = False

## Функция Розенброка

In [15]:
def rosenbrock_func(a: float, b: float, f0: float, n: int) -> Callable:
    def rosenbrock_func_wrapper(x: np.ndarray) -> float:
        return sum(a * (x[i] ** 2 - x[i + 1]) ** 2 + b * (x[i] - 1) ** 2 for i in range(n - 1)) + f0
    return rosenbrock_func_wrapper

## Градиент функции Розенброка

In [16]:
def rosenbrock_func_grad(a: float, b: float, f0: float, n: int) -> Callable:
    def rosenbrock_func_grad_wrapper(x: np.ndarray) -> np.ndarray:
        grad = []
        df_dx0 = 4 * a * x[0] * (x[0] ** 2 - x[1]) + 2 * b * (x[0] - 1)
        grad.append(df_dx0)
        for i in range(1, n - 1):
            df_dxi = - 2 * a * (x[i - 1] ** 2 - x[i]) + 4 * a * x[i] * (x[i] ** 2 - x[i + 1]) + 2 * b * (x[i] - 1)
            grad.append(df_dxi)
        df_dxn_add_1 = - 2 * a * (x[n - 2] ** 2 - x[n - 1])
        grad.append(df_dxn_add_1)
        return np.array(grad)
    return rosenbrock_func_grad_wrapper

## Гессиан функции Розенброка

In [17]:
def rosenbrock_func_hessian(a: float, b: float, f0: float, n: int) -> Callable:
    def rosenbrock_func_hessian_wrapper(x: np.ndarray) -> np.ndarray:
        H = []
        df_dx0_dx0 = 4 * a * (x[0] ** 2 - x[1]) + 8 * a * x[0] ** 2 + 2 * b
        df_dx0_dx1 = - 4 * a * x[0]
        H.append([df_dx0_dx0, df_dx0_dx1] + [0 for _ in range(n - 2)])
        for i in range(1, n - 1):
            df_dxi_dxi_min1 = - 4 * a * x[i - 1] - 4 * a * x[i]
            df_dxi_dxi = 2 * a + 12 * a * x[i] ** 2 + 2 * b
            df_dxi_dxi_add_1 = - 4 * a * x[i]
            H.append([0 for _ in range(i - 2)] + [df_dxi_dxi_min1, df_dxi_dxi, df_dxi_dxi_add_1] + [0 for _ in range(i + 2, n)])
        H.append([0 for _ in range(n - 2)] + [-4 * a * x[n - 2], 2 * a])
        return np.array(H)
    return rosenbrock_func_hessian_wrapper

## Равномерная норма вектора

In [18]:
def vector_norm(vec: np.ndarray) -> float:
    return max(map(abs, vec))

## Метод одномерного поиска

In [19]:
def find_min(func: Callable, a: float, b: float, xk: np.ndarray, dk: np.ndarray, eps: float=1e-4):
    phi = 1.618
    a1 = b - (b - a) / phi
    b1 = a + (b - a) / phi

    while abs(b1 - a1) > eps:
        if func(xk + a1 * dk) < func(xk + b1 * dk):
            b = b1
        else:
            a = a1
            
        a1 = b - (b - a) / phi
        b1 = a + (b - a) / phi

    return (a + b) / 2 

## Метод Флетчера–Ривза

In [20]:
def fletcher_reeves(func: Callable, func_grad: np.ndarray, x0: np.ndarray, eps: float=1e-5) -> tuple[np.ndarray, float, int]:
    dk = -func_grad(x0)
    xk = x0.copy()
    k = 0
    while True:
        if vector_norm(func_grad(xk)) < eps:
            break
        # используем метод одномерной оптимизации для поиска минимума функции
        alpha = find_min(func, -1000, 1000, xk, dk, 1e-4)
        if alpha > 1e-4:
            xk1 = xk + alpha * dk
        wk = (vector_norm(func_grad(xk1)) ** 2) / (vector_norm(func_grad(xk)) ** 2)
        dk = -func_grad(xk1) + wk * dk
        xk = xk1.copy()
        k += 1
    return xk, func(xk), k

## Метод Полака-Рибьера

In [21]:
def polack_ribeir(func: Callable, func_grad: np.ndarray, x0: np.ndarray, eps: float=1e-5) -> tuple[np.ndarray, float, int]:
    n = len(x0)
    dk = -func_grad(x0)
    xk = x0.copy()
    k = 0
    while True:
        if vector_norm(func_grad(xk)) < eps:
            break
        # используем метод одномерной оптимизации для поиска минимума функции
        alpha = find_min(func, -1000, 1000, xk, dk, 1e-4)
        if alpha > 1e-4:
            xk1 = xk + alpha * dk
        if k % n == 0:
            wk = np.dot(func_grad(xk1), (func_grad(xk1) - func_grad(xk))) / (vector_norm(func_grad(xk)) ** 2)
        else:
            wk = 0
        dk = -func_grad(xk1) + wk * dk
        xk = xk1.copy()
        k += 1
    return xk, func(xk), k

## Метод Девидона–Флетчера–Пауэлла

In [22]:
def davidon_fletcher_powell(func: Callable, func_grad: np.ndarray, x0: np.ndarray, eps=1e-5) -> tuple[np.ndarray, float, int]:
    Gk = np.eye(len(x0))
    xk = x0.copy()
    k = 0
    while True:
        if vector_norm(func_grad(xk)) < eps:
            break
        dk = np.dot(-Gk, func_grad(xk))
        # используем метод одномерной оптимизации для поиска минимума функции
        alpha = find_min(func, -1000, 1000, xk, dk, 1e-4)
        if alpha > 1e-4:
            xk1 = xk + alpha * dk
        delta_xk = xk1 - xk
        delta_gk = func_grad(xk1) - func_grad(xk)
        delta_Gk = np.outer(delta_xk, delta_xk) / (np.dot(np.transpose(delta_xk), delta_gk)) \
                - np.outer(np.dot(Gk, delta_gk), np.dot(Gk, delta_gk))  \
                / (np.dot(np.transpose(np.dot(Gk, delta_gk)), delta_gk))
        Gk = Gk + delta_Gk
        xk = xk1.copy()
        k += 1
    return xk, func(xk), k

## Метод Левенберга–Марквардта

In [23]:
def levenberg_marquardt(func: Callable, func_grad: np.ndarray, H: np.ndarray, x0: np.ndarray, eps=1e-5) -> tuple[np.ndarray, float, int]:
    xk = x0.copy()
    muk = 1e1
    k = 0
    while True:
        if vector_norm(func_grad(xk)) < eps:
            break
        Hxk = H(xk)
        xk1 = xk - np.dot(np.linalg.inv(Hxk + muk * np.eye(Hxk.shape[0], Hxk.shape[1])), func_grad(xk))
        if func(xk1) < func(xk):
            k += 1
            muk = muk / 2
            xk = xk1.copy()
        else:
            muk = 2 * muk
    return xk, func(xk), k

## Тестирование

In [24]:
a, b, f0, n = 10, 250, 45, 3
x0 = np.random.uniform(-10, 10, n)
func = rosenbrock_func(a, b, f0, n)
func_grad = rosenbrock_func_grad(a, b, f0, n)
H = rosenbrock_func_hessian(a, b, f0, n)
data = {
    'fletcher_reeves': [],
    'polack_ribeir': [],
    'davidon_fletcher_powell': [],
    'levenberg_marquardt': []
}
for eps in (1e-2, 1e-3, 1e-4, 1e-5):
    print(f'{eps = }')
    it_nums = []
    for method in (fletcher_reeves, polack_ribeir, davidon_fletcher_powell, levenberg_marquardt):
        x, y, it_num = method(func, func_grad, x0, eps) if method.__name__ != 'levenberg_marquardt' else method(func, func_grad, H, x0, eps)
        data[f'{method.__name__}'].append(it_num)
        print(method.__name__, x, y, it_num, sep='\n', end='\n\n')

eps = 0.01
fletcher_reeves
[1.00000059 0.99999825 0.99997758]
45.000000004519755
21

polack_ribeir
[1.00000668 1.00000775 1.00008769]
45.00000007860819
18

davidon_fletcher_powell
[0.99998576 0.9999973  1.00005325]
45.000000093555784
7

levenberg_marquardt
[0.99999961 0.99999441 0.99998682]
45.00000000813164
7

eps = 0.001
fletcher_reeves
[1.00000059 0.99999825 0.99997758]
45.000000004519755
21

polack_ribeir
[0.99999985 1.00000302 1.0000353 ]
45.000000010955915
21

davidon_fletcher_powell
[0.99999998 0.99999997 0.99999977]
45.000000000000604
8

levenberg_marquardt
[0.99999997 0.99999962 0.99999919]
45.00000000003733
8

eps = 0.0001
fletcher_reeves
[0.99999997 1.00000002 0.9999985 ]
45.00000000002445
31

polack_ribeir
[1.         1.0000001  0.99999992]
45.00000000000365
26

davidon_fletcher_powell
[0.99999998 0.99999997 0.99999977]
45.000000000000604
8

levenberg_marquardt
[1.         0.99999997 0.99999995]
45.00000000000017
9

eps = 1e-05
fletcher_reeves
[1.         1.00000005 1.00000

In [25]:
df = pd.DataFrame(data=data, index=['1e-2', '1e-3', '1e-4', '1e-5'])
df.index.name = 'eps'
styled_df = df.style.set_table_styles([{'selector': '', 'props': [('border', '2px solid black')]}])\
.set_properties(**{'border': '2px solid black'})
styled_df

,fletcher_reeves,polack_ribeir,davidon_fletcher_powell,levenberg_marquardt
eps,,,,
1e-2,21,18,7,7
1e-3,21,21,8,8
1e-4,31,26,8,9
1e-5,40,27,9,10


Метод Левенберга-Марквардта показал наибольшую устойчивость среди рассмотренных методов и очень высокую скорость сходимости. Однако данный метод налагает ограничения на целевую функцию (ограниченность снизу, наличие непрерывных вторых частных производных), а также довольно сложен в реализации.

## Генетический алгоритм

In [26]:
def fitness_func(x: np.ndarray):
    return 1 / func(x)

In [64]:
def genetic_algorithm(func: Callable, n: int) -> tuple[np.ndarray, float]:
    t, Np, k, Mp = 0, 10, 1, 100
    # формируем исходную популяцию с количеством особей Mp
    population = np.array([np.random.uniform(0, 1, n) for _ in range(Mp)])
    fitness_values_individuals = np.array(list(map(fitness_func, population)))
    fitness_value_population = np.sum(fitness_values_individuals)

    while t != Np:
        k = 1
        while k < Mp:
            # этап селекции (отбора особей) для последующего их скрещивания
            q = fitness_values_individuals / fitness_value_population
            new_population = [population[np.random.choice(len(population), p=q)] for _ in range(Mp)]

            # этап скрещивания
            Pc = 0.4
            parent_indices = []
            for i in range(len(new_population)):
                r = np.random.random()
                if r < Pc:
                    parent_indices.append(i)

            for parent1_index, parent2_index in zip(parent_indices[0::2], parent_indices[1::2]):
                c = np.random.random()
                parent1, parent2 = new_population[parent1_index], new_population[parent2_index]
                parent1 = c * parent1 + (1 - c) * parent2
                parent2 = (1 - c) * parent1 + c * parent2
                
            # этап мутации
            Pm = 0.2
            for i in range(len(new_population)):
                r = np.random.random()
                if r < Pm:
                    p = np.random.randint(0, n, 1)
                    new_population[i][p] = np.random.uniform(0, 1)

            population[:] = new_population
            fitness_values_individuals = np.array(list(map(fitness_func, population)))
            fitness_value_population = np.sum(fitness_values_individuals)
            k += 1
        t += 1
    best_individual = population[np.argmax(fitness_values_individuals)]
    return best_individual, func(best_individual)

In [72]:
x, y = genetic_algorithm(func, n)
print(x, y)

[0.96546144 0.96752585 0.87007255] 45.6180139124816
